In [17]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np

In [18]:
start_year = datetime.now().year
end_year = start_year + 1
training_year = 4

In [19]:
market = Market()
speculation_db = ADatabase("bitcoin_speculation")

In [1]:
speculation_db.connect()
data = speculation_db.retrieve("pdr_data")
speculation_db.disconnect()

NameError: name 'market' is not defined

In [22]:
factors = [str(x) for x in range(14)]
factors.append("adjclose")
factors.append("d1")
factors.append("d2")
factors.append("d3")

In [24]:
sim = []
speculation_db.connect()
errors = []
for year in tqdm(range(start_year,end_year)):
    try:
        training_data = data[(data["year"]<year) & (data["year"]>=year-training_year)]
        training_data.dropna(inplace=True)
        training_data.reset_index(drop=True,inplace=True)
        prediction_data = data[data["year"]==year].dropna().reset_index()
        if training_data.index.size > 100 and prediction_data.index.size > 40:
            refined = {"X":training_data[factors],"y":training_data["y"]}
            models = m.regression(refined)            
            models["year"] = year
            models["training_year"] = training_year
            models["model"] = [pickle.dumps(x) for x in models["model"]]
            speculation_db.store("models",models)
        else:
            errors.append([year,training_data.index.size,prediction_data.index.size,"not_enough_stuff"])
    except Exception as e:
        errors.append([year,training_data.index.size,prediction_data.index.size,str(e)])
speculation_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:11<00:00, 10.27s/it]
